In [ ]:
import os
import requests
import utils
import pandas as pd
import matplotlib.pyplot as plt
from lol_backend_service import RiotAPI
from constants import Clusters
from dotenv import load_dotenv

load_dotenv()

In [ ]:
riot_api = RiotAPI(os.getenv("PERSONAL_API_KEY"))
game_name = os.getenv("PERSONAL_GAME_NAME")
tagline = str(os.getenv("PERSONAL_TAGLINE"))

## Getting the feel of the API

### Get PUUID

In [ ]:
player_puuid = riot_api.get_puuid_by_riot_id(game_name, tagline)
player_puuid

### Get Free Champion Rotation

In [ ]:
curr_free_champ = utils.get_champion_mapping(riot_api.get_free_champ_rotation('PH2'))
curr_free_champ

## Data fetching

### Convert match data to dataframe

In [ ]:
match_list = riot_api.get_match_list("SEA", player_puuid, count=90)

In [ ]:
player_match_list = []
df_player_matches = pd.DataFrame()
if os.path.exists(f'./player_matches/{game_name}_{tagline}_matches.csv'):
    df_player_matches = pd.read_csv(f'./player_matches/{game_name}_{tagline}_matches.csv')
else:
    for match_id in match_list:
        match_data = riot_api.get_match_data("SEA", match_id)
        match_game_mode = match_data['info']['gameMode']

        match_data = utils.extract_player_match_data(match_data, game_name, tagline)
        match_data = {x: match_data[x] for x in match_data if x not in ['challenges', 'missions', 'perks']}
        match_data = pd.Series(match_data)

        match_data['matchId'] = match_id
        match_data['gameMode'] = match_game_mode

        player_match_list.append(match_data)

    df_player_matches = pd.DataFrame(player_match_list)
    df_player_matches.to_csv(f'./player_matches/{game_name}_{tagline}_matches.csv', index=False)

## Data Analysis

### Grouping columns for Data Analysis

In [ ]:
metadata_list = ['matchId', 'participants']
combat_list = ['matchId', 'gameMode', 'puuid', 'summonerName', 'teamPosition', 'kills', 'deaths', 'assists', 'firstBloodKill', 'firstBloodAssist', 'goldEarned', 'totalDamageDealt', 'totalDamageDealtToChampions', 'totalDamageTaken', 'totalMinionsKilled', 'visionScore', 'win']
ping_list = ['matchId', 'puuid', 'summonerName', 'allInPings', 'assistMePings', 'basicPings', 'commandPings', 'dangerPings', 'enemyMissingPings', 'enemyVisionPings', 'getBackPings', 'holdPings', 'needVisionPings', 'onMyWayPings', 'pushPings', 'retreatPings', 'visionClearedPings']

### Combat Analysis

In [ ]:
df_combat = df_player_matches[combat_list]
df_combat.head()

In [ ]:
df_combat.describe()

### Descriptive Analysis for each player

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
barh_plot = ax.barh(df_combat['gameMode'].value_counts().index, df_combat['gameMode'].value_counts().values)
ax.set_title('Game Mode Distribution')
ax.bar_label(barh_plot)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

win_count = df_combat['win'].value_counts()
win_count.index = win_count.index.map({True: 'Win', False: 'Loss'})

barh_plot = ax.barh(win_count.index, win_count.values)
ax.set_title('Win Distribution')

ax.bar_label(barh_plot)
plt.show()

In [ ]:
# plot normal distribution of gold earned
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(df_combat['goldEarned'])
ax.set_title('Gold Earned Distribution')
plt.show()

In [ ]:
# plot normal distribution using seaborn
import seaborn as sns

fig, ax = plt.subplots(figsize=(10, 6))
sns.histplot(df_combat['goldEarned'], kde=True, ax=ax)
ax.set_title('Gold Earned Distribution')
plt.show()

### Ping Analysis (for fun)

In [ ]:
df_player_matches[ping_list].head()
df_player_matches[ping_list].to_csv(f'./player_matches/{game_name}_{tagline}_pings.csv', index=False)